In [5]:
import pandas as pd
import numpy as np
from IPython import display as ICD


from pickle import load, dump
from scripts.preprocess import load_index_data, load_tbills_data
from scripts.utils import print_results

# Full

## Boostrap

In [3]:
bootstrap = load(open('../../res_bootstrap_unpacked.pkl', 'rb'))
values = pd.DataFrame([sample.loc['value'] for sample in bootstrap.values()], index=range(len(bootstrap)))
boot_res = values.describe().loc[['mean', 'std']]

In [5]:
res = load(open('../../res.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo').iloc[:,0].values

implied, params = print_results(pred, log_mk, log_rf, False)
ICD.display(pred)
ICD.display(implied)
ICD.display(params)

,gamma,delta,sigma,k,a,b,pi_err
value,0.056164,-1.056373,0.195405,0.538005,0.522499,10.000180,0.276883
std,0.000289,0.002290,0.003481,0.005971,0.001495,0.000916,0.041095


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.952746,33.302074,32.291673,-1.131642
std,40.088604,42.146657,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.465588,-1.056373,39.081091,53.800463,0.522499,10.000180,27.688293
sd,0.115474,0.002290,0.696293,0.321248,0.000781,0.000916,0.822800


## Cochrane

In [32]:
res = load(open('./results/res_cochrane.pkl', 'rb'))
start, end = '1987-01-01', '2000-08-02'
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', test=True, prefix='../../').values

implied, params = print_results(res, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)


# Cochrane
# -2.12  0.96   99.23   16.04  0.76   4.76   1.26
# 1.09   0.09   1.75    0.79   0.02   0.13   0.9

Using parameters (annualized percentages)
E[log Rf]=5.26%, E[log Rm]=15.92%, V[log Rm]=14.80%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,34.308917,90.608776,71.6657,1.129408
std,97.328385,114.497715,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,18.808555,0.960933,96.283223,17.245886,0.748421,4.807627,5.765190
sd,1.054837,0.078594,2.167803,0.515302,0.021466,0.115496,0.572347


## Baseline

In [36]:
res = load(open('./results/res.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.45%, E[log Rm]=3.38%, V[log Rm]=8.46%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.935279,33.283870,32.292905,-1.131543
std,40.090256,42.147543,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.465588,-1.056373,39.081091,53.800463,0.522499,10.000180,27.688293
sd,0.115474,0.002290,0.696293,0.321248,0.000781,0.000916,0.822800


## Industries

In [37]:
res = load(open('./results/res_Health.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.45%, E[log Rm]=3.38%, V[log Rm]=8.46%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.996705,50.746309,50.138498,-1.426878
std,68.962810,75.437620,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.735532,-1.280739,68.106013,15.812377,0.571673,9.999444,9.937278
sd,1.201222,0.184304,3.330918,0.337230,0.004904,0.000812,2.878878


In [38]:
res = load(open('./results/res_Other.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.30%, E[log Rm]=3.31%, V[log Rm]=8.39%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,16.602002,19.292409,18.041192,-0.766489
std,21.164118,21.729294,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,15.052120,-0.739078,20.235602,86.947560,0.524608,9.992784,67.164687
sd,0.443088,0.049559,7.391515,1.063108,0.002278,0.000824,3.090092


In [39]:
res = load(open('./results/res_Retail.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.19%, E[log Rm]=3.36%, V[log Rm]=8.64%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,20.528945,29.067261,27.928585,-0.683958
std,38.809052,40.575767,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,19.087538,-0.644586,38.407755,59.136139,0.529450,9.986410,27.906953
sd,0.384297,0.067948,4.335142,1.597876,0.002101,0.000933,2.156463


In [40]:
res = load(open('./results/res_Tech.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.31%, E[log Rm]=3.36%, V[log Rm]=8.42%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,21.810904,29.105371,28.428608,-1.156679
std,35.452167,37.009672,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.648871,-1.090301,34.243466,61.838131,0.516370,9.998979,31.640361
sd,0.188914,0.020892,1.646535,0.339479,0.000994,0.000193,0.812557


## Before 2000

In [26]:
res = load(open('./results/res_2000.pkl', 'rb'))
start, end = '1987-01-01', '2000-08-02'
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(res, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=5.26%, E[log Rm]=5.42%, V[log Rm]=7.60%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,101.209273,529.746409,533.417333,-19.798031
std,217.333686,458.100872,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,97.325745,-8.700604,207.043480,9.641510,1.774183,2.101270,3.546142
sd,17.461552,0.597502,19.472328,3.485849,0.413910,0.259416,4.788300


In [27]:
res

,gamma,delta,sigma,k,a,b,pi_err
value,0.243314,-8.700604,1.035217,0.096415,1.774183,2.101270,0.035461
std,0.043654,0.597502,0.097362,0.361546,0.233296,0.259416,1.399928


# From 2010

## Baseline

In [44]:
res = load(open('../results/res_from_2010.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.044147,38.597624,32.143804,1.369221
std,41.959509,44.425256,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.486135,1.263586,40.741357,58.001268,0.524208,9.976762,25.248838
sd,0.561361,0.131019,1.632983,0.542034,0.001035,0.000852,0.753908


## Industries

In [46]:
res = load(open('../results/res_from_2010_tech.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,30.447771,44.398815,36.028574,1.808849
std,48.281191,51.640742,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.350359,1.647008,46.474905,52.761381,0.535861,9.994694,22.433772
sd,0.573147,0.096091,2.414756,1.118730,0.001282,0.000661,1.117114


In [47]:
res = load(open('../results/res_from_2010_other.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.049506,36.410122,33.125152,0.642284
std,48.575001,51.495145,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.171717,0.596017,48.343735,39.852130,0.493348,9.999882,14.341468
sd,0.410862,0.011073,2.297246,0.801886,0.001529,0.000870,1.007226


In [48]:
res = load(open('../results/res_from_2010_health.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,24.738513,28.837675,24.074829,0.98131
std,24.928114,25.911634,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.534031,0.926460,23.817091,85.994521,0.545547,9.999827,54.834035
sd,1.589694,0.209106,17.892963,4.103842,0.005613,0.003894,5.782390


In [49]:
res = load(open('../results/res_from_2010_retail.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.460272,38.650506,30.048132,1.862101
std,38.561903,40.760224,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.079368,1.717618,36.067626,63.138034,0.532096,9.996947,28.519121
sd,0.988565,0.081985,6.428220,1.994004,0.003466,0.000788,3.203769


## NASDAQ